# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

#configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Reading from csv into dataframe using pandas
weatherdataframe = pd.read_csv("../WeatherPy/Weather CSV")
weatherdataframe.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,74.62,81,100,20.00,PF,1603079327
1,1,Jamestown,42.10,-79.24,55.00,62,90,10.29,US,1603079457
2,2,Beyneu,45.32,55.20,55.72,34,0,13.62,KZ,1603079594
3,3,Ushuaia,-54.80,-68.30,37.40,74,20,16.11,AR,1603079595
4,4,Devils Lake,48.11,-98.87,28.40,50,90,12.75,US,1603079409


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Store latitude and longitude in locations:
locations = weatherdataframe[["Lat", "Lng"]]

#Store humidity values. Fill NaN values and convert to float.

humidity = weatherdataframe["Humidity"].astype(float)

#Calculating the maximum humidity for plotting:
humidity_max = humidity.max()
humidity_max

100.0

In [4]:
#Plot the heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = humidity_max, point_radius = 1)

#Add the layer to the map
fig.add_layer(heat_layer)

#Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrowed dataframe to find ideal weather condition: Temp
idealtemp_df = weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70)]

In [6]:
#Narrowed dataframe to find ideal weather condition: Wind Speed
idealwind_df = weatherdataframe.loc[weatherdataframe["Wind Speed"]<10]

In [7]:
#Narrowed dataframe to find ideal weather condition: Zero Cloudiness
idealwind_df = weatherdataframe.loc[weatherdataframe["Cloudiness"]==0]

In [8]:
#Combined all conditions into a single dataframe:

allconditionslong_df= weatherdataframe.loc[(weatherdataframe["Max Temp"]<80) & (weatherdataframe["Max Temp"]>70) & (weatherdataframe["Wind Speed"]<10) & (weatherdataframe["Cloudiness"]==0)]

#Setting limit of 10 results
allconditions_df=allconditionslong_df.head(10)

allconditions_df





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,Kiryat Gat,31.61,34.76,71.01,86,0,2.95,IL,1603079520
151,151,Oodweyne,9.41,45.06,72.75,42,0,4.65,SO,1603079754
229,229,Cayenne,4.93,-52.33,75.20,100,0,8.99,GF,1603079838
243,243,Abu Dhabi,24.47,54.37,77.00,44,0,2.24,AE,1603079589
277,277,Taoudenni,22.68,-3.98,76.03,19,0,4.79,ML,1603079890
303,303,Bilma,18.69,12.92,76.71,19,0,3.87,NE,1603079919
320,320,Aswān,24.09,32.91,76.64,24,0,6.13,EG,1603079938
362,362,Grand Gaube,-20.01,57.66,78.01,65,0,7.00,MU,1603079937
434,434,Makkah al Mukarramah,21.43,39.83,77.04,18,0,2.24,SA,1603080073
445,445,Ghanzi,-21.57,21.78,72.14,20,0,6.96,BW,1603080085


In [9]:
allconditions_df.count()

Unnamed: 0    10
City          10
Lat           10
Lng           10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
Date          10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#New hotel dataframe
hotel_df = allconditions_df[["City","Lat","Lng","Country"]].copy()

In [11]:
#Creating new hotel name column for dataframe
hotel_df["Hotel Name"]= ""
hotel_df

,City,Lat,Lng,Country,Hotel Name
17,Kiryat Gat,31.61,34.76,IL,
151,Oodweyne,9.41,45.06,SO,
229,Cayenne,4.93,-52.33,GF,
243,Abu Dhabi,24.47,54.37,AE,
277,Taoudenni,22.68,-3.98,ML,
303,Bilma,18.69,12.92,NE,
320,Aswān,24.09,32.91,EG,
362,Grand Gaube,-20.01,57.66,MU,
434,Makkah al Mukarramah,21.43,39.83,SA,
445,Ghanzi,-21.57,21.78,BW,


In [12]:
#base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#creating parameters dictionary    
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # Get coordinate data type from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    #Get name data from df
    city = row["City"]

    # Add location to params dictionary (necessary for nearby)
    params["location"] = f"{latitude},{longitude}"
    
    # Assemble url, make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Nearby hotel in {city}: {results[0]['name']}.")
        
        #Putting this iterated info into a dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    #Break
    time.sleep(1)


Retrieving Results for Index 17: Kiryat Gat.
Missing field/result... skipping.
Retrieving Results for Index 151: Oodweyne.
Missing field/result... skipping.
Retrieving Results for Index 229: Cayenne.
Missing field/result... skipping.
Retrieving Results for Index 243: Abu Dhabi.


KeyboardInterrupt: 

In [ ]:
#Checking if hotels printed
hotel_df

In [ ]:
#Creating hotel_info dataframe
#hotel_info=[["City","Country","Hotel Name"]]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map, use hotel_info to fill the info box
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig